In [ ]:
# If we have caliper, then we can limit the spinner flow rate analysis to only those intervals that are in gauge

# Jorge Acuna paper at stanford workshop

# auto removal of data that are inconsistant with what would be expected (for instance the spinner was stuck)

You have finished the T21 geothermal well completion test tutoral. Well done!

<hr />

© 2021 [CC-BY](https://creativecommons.org/licenses/by/4.0/) [Irene Wallis](https://www.cubicearth.nz/) and Katie McLean 
